# DermLIP 모델을 이용한 피부 질환 진단 정확도 평가

이 노트북은 DermLIP (Dermatology Language-Image Pretraining) 모델을 사용하여 피부 질환 이미지에 대한 진단 정확도를 평가합니다.

## 모델 정보
- **DermLIP-ViT-B/16**: Vision Transformer 기반 모델
- **DermLIP-PanDerm**: PanDerm 아키텍처 기반 (최고 성능)
- **훈련 데이터**: Derm1M (1,029,761 이미지-텍스트 쌍, 390개 피부 질환)

## 참고 문헌
- GitHub: https://github.com/SiyuanYan1/Derm1M
- Hugging Face: redlessone/DermLIP_ViT-B-16, redlessone/DermLIP_PanDerm-base-w-PubMed-256

## 1. 필수 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 실행시에만)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install open_clip_torch
!pip install pillow
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import open_clip
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from tqdm import tqdm
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 중인 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. DermLIP 모델 로드

In [ ]:
class DermLIPModel:
    """DermLIP 모델을 로드하고 추론하는 클래스"""
    
    def __init__(self, model_name='hf-hub:redlessone/DermLIP_ViT-B-16', device='cuda'):
        """
        Args:
            model_name: 사용할 모델 이름
                - 'hf-hub:redlessone/DermLIP_ViT-B-16' (기본)
                - 'hf-hub:redlessone/DermLIP_PanDerm-base-w-PubMed-256' (최고 성능)
            device: 'cuda' 또는 'cpu'
        """
        self.device = device
        self.model_name = model_name
        
        print(f'모델 로드 중: {model_name}...')
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=model_name,
            device=device
        )
        self.tokenizer = open_clip.get_tokenizer(model_name)
        
        self.model.eval()
        print('모델 로드 완료!')
    
    def encode_images(self, images):
        """이미지를 인코딩하여 임베딩 벡터 반환"""
        with torch.no_grad():
            image_features = self.model.encode_image(images)
            image_features = F.normalize(image_features, dim=-1)
        return image_features
    
    def encode_texts(self, texts):
        """텍스트를 인코딩하여 임베딩 벡터 반환"""
        tokens = self.tokenizer(texts).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(tokens)
            text_features = F.normalize(text_features, dim=-1)
        return text_features
    
    def predict(self, image, class_descriptions):
        """
        이미지에 대한 예측 수행 (Zero-shot Classification)
        
        Args:
            image: PIL Image 또는 전처리된 텐서
            class_descriptions: 클래스별 텍스트 설명 리스트
        
        Returns:
            predicted_class_idx: 예측된 클래스 인덱스
            probabilities: 각 클래스에 대한 확률
        """
        # 이미지 전처리
        if isinstance(image, Image.Image):
            image = self.preprocess(image).unsqueeze(0).to(self.device)
        
        # 이미지와 텍스트 인코딩
        image_features = self.encode_images(image)
        text_features = self.encode_texts(class_descriptions)
        
        # 유사도 계산 (cosine similarity)
        similarity = (image_features @ text_features.T).squeeze(0)
        probabilities = F.softmax(similarity * 100, dim=0)  # temperature scaling
        
        predicted_class_idx = torch.argmax(probabilities).item()
        
        return predicted_class_idx, probabilities.cpu().numpy()

In [ ]:
# DermLIP 모델 로드
# 옵션 1: ViT-B/16 모델 (빠름)
model = DermLIPModel(
    model_name='hf-hub:redlessone/DermLIP_ViT-B-16',
    device=device
)

# 옵션 2: PanDerm 모델 (최고 성능, 더 느림)
# model = DermLIPModel(
#     model_name='hf-hub:redlessone/DermLIP_PanDerm-base-w-PubMed-256',
#     device=device
# )

## 3. 데이터셋 설정

자신의 피부 질환 이미지 데이터를 준비합니다.

### 데이터 구조 예시:
```
data/
├── images/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── labels.csv  # columns: image_path, label
```

In [ ]:
class SkinDiseaseDataset(Dataset):
    """피부 질환 이미지 데이터셋"""
    
    def __init__(self, data_df, preprocess_fn):
        """
        Args:
            data_df: DataFrame with columns ['image_path', 'label']
            preprocess_fn: 이미지 전처리 함수
        """
        self.data_df = data_df
        self.preprocess_fn = preprocess_fn
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        
        # 이미지 로드
        image_path = row['image_path']
        image = Image.open(image_path).convert('RGB')
        image = self.preprocess_fn(image)
        
        # 레이블
        label = row['label']
        
        return image, label, image_path

In [ ]:
# 피부 질환 클래스 정의 (예시)
# 실제 데이터에 맞게 수정하세요
SKIN_DISEASE_CLASSES = {
    0: 'acne',
    1: 'eczema',
    2: 'psoriasis',
    3: 'melanoma',
    4: 'basal cell carcinoma',
    5: 'seborrheic keratosis',
    6: 'rosacea',
    7: 'vitiligo',
    8: 'herpes',
    9: 'warts'
}

# 클래스별 설명 (Zero-shot classification에 사용)
# DermLIP 모델은 임상적 설명과 함께 훈련되었으므로 상세한 설명이 효과적입니다
CLASS_DESCRIPTIONS = [
    "a photo of acne, inflammatory skin condition with pimples and blackheads",
    "a photo of eczema, red itchy inflamed skin condition",
    "a photo of psoriasis, skin disease with red scaly patches",
    "a photo of melanoma, malignant skin cancer with irregular pigmented lesion",
    "a photo of basal cell carcinoma, common skin cancer with pearly bump",
    "a photo of seborrheic keratosis, benign brown growth on the skin",
    "a photo of rosacea, facial redness and visible blood vessels",
    "a photo of vitiligo, loss of skin pigmentation with white patches",
    "a photo of herpes, viral infection causing painful blisters",
    "a photo of warts, small rough growths caused by human papillomavirus"
]

print(f'클래스 개수: {len(SKIN_DISEASE_CLASSES)}')
print('\n클래스 목록:')
for idx, name in SKIN_DISEASE_CLASSES.items():
    print(f'  {idx}: {name}')

In [ ]:
# 데이터 로드 예시
# 실제 데이터 경로로 수정하세요

# 방법 1: CSV 파일에서 로드
# data_df = pd.read_csv('data/labels.csv')
# data_df의 형식: columns=['image_path', 'label']

# 방법 2: 폴더 구조에서 로드
# data_dir = 'data/images/'
# data_list = []
# for class_idx, class_name in SKIN_DISEASE_CLASSES.items():
#     class_dir = os.path.join(data_dir, class_name)
#     if os.path.exists(class_dir):
#         for img_file in os.listdir(class_dir):
#             if img_file.endswith(('.jpg', '.jpeg', '.png')):
#                 data_list.append({
#                     'image_path': os.path.join(class_dir, img_file),
#                     'label': class_idx
#                 })
# data_df = pd.DataFrame(data_list)

# 예시 데이터 (실제 사용시 삭제하고 위의 코드 사용)
print("주의: 실제 데이터를 로드하려면 위의 코드를 주석 해제하고 경로를 수정하세요.")
print("\n현재는 예시 코드입니다.")

# data_df가 준비되었다면:
# print(f'\n총 이미지 개수: {len(data_df)}')
# print(f'클래스별 분포:\n{data_df["label"].value_counts().sort_index()}')

## 4. 진단 정확도 평가

In [ ]:
def evaluate_model(model, data_df, class_descriptions, batch_size=32):
    """
    모델의 진단 정확도를 평가합니다.
    
    Args:
        model: DermLIPModel 인스턴스
        data_df: 평가 데이터 DataFrame
        class_descriptions: 클래스 설명 리스트
        batch_size: 배치 크기
    
    Returns:
        results: 평가 결과 딕셔너리
    """
    dataset = SkinDiseaseDataset(data_df, model.preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    print('평가 시작...')
    
    for images, labels, image_paths in tqdm(dataloader, desc='평가 중'):
        images = images.to(model.device)
        
        # 배치 예측
        image_features = model.encode_images(images)
        text_features = model.encode_texts(class_descriptions)
        
        # 유사도 계산
        similarity = image_features @ text_features.T
        probabilities = F.softmax(similarity * 100, dim=1)
        predictions = torch.argmax(probabilities, dim=1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.numpy())
        all_probabilities.extend(probabilities.cpu().numpy())
    
    # 평가 지표 계산
    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_predictions, average='weighted', zero_division=0
    )
    
    # 클래스별 정확도
    class_report = classification_report(
        all_labels, all_predictions,
        target_names=[SKIN_DISEASE_CLASSES[i] for i in range(len(SKIN_DISEASE_CLASSES))],
        output_dict=True,
        zero_division=0
    )
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities,
        'confusion_matrix': cm,
        'classification_report': class_report
    }
    
    return results

In [ ]:
# 모델 평가 실행
# 실제 데이터가 준비되면 주석 해제하여 실행

# results = evaluate_model(
#     model=model,
#     data_df=data_df,
#     class_descriptions=CLASS_DESCRIPTIONS,
#     batch_size=32
# )

# print('\n=== 평가 결과 ===')
# print(f'정확도 (Accuracy): {results["accuracy"]:.4f} ({results["accuracy"]*100:.2f}%)')
# print(f'정밀도 (Precision): {results["precision"]:.4f}')
# print(f'재현율 (Recall): {results["recall"]:.4f}')
# print(f'F1 Score: {results["f1_score"]:.4f}')

## 5. 결과 시각화

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """Confusion Matrix 시각화"""
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )
    plt.title('Confusion Matrix', fontsize=16, pad=20)
    plt.ylabel('실제 클래스', fontsize=12)
    plt.xlabel('예측 클래스', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_class_performance(class_report, class_names):
    """클래스별 성능 시각화"""
    metrics_df = pd.DataFrame({
        'Class': class_names,
        'Precision': [class_report[name]['precision'] for name in class_names],
        'Recall': [class_report[name]['recall'] for name in class_names],
        'F1-Score': [class_report[name]['f1-score'] for name in class_names]
    })
    
    fig, ax = plt.subplots(figsize=(14, 6))
    x = np.arange(len(class_names))
    width = 0.25
    
    ax.bar(x - width, metrics_df['Precision'], width, label='Precision', alpha=0.8)
    ax.bar(x, metrics_df['Recall'], width, label='Recall', alpha=0.8)
    ax.bar(x + width, metrics_df['F1-Score'], width, label='F1-Score', alpha=0.8)
    
    ax.set_xlabel('클래스', fontsize=12)
    ax.set_ylabel('점수', fontsize=12)
    ax.set_title('클래스별 성능 지표', fontsize=16, pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('class_performance.png', dpi=300, bbox_inches='tight')
    plt.show()

def visualize_predictions(model, data_df, class_descriptions, num_samples=9):
    """예측 결과 샘플 시각화"""
    sample_indices = np.random.choice(len(data_df), min(num_samples, len(data_df)), replace=False)
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.ravel()
    
    for idx, sample_idx in enumerate(sample_indices):
        row = data_df.iloc[sample_idx]
        image_path = row['image_path']
        true_label = row['label']
        
        # 이미지 로드 및 예측
        image = Image.open(image_path).convert('RGB')
        pred_idx, probabilities = model.predict(image, class_descriptions)
        
        # 시각화
        axes[idx].imshow(image)
        axes[idx].axis('off')
        
        true_class = SKIN_DISEASE_CLASSES[true_label]
        pred_class = SKIN_DISEASE_CLASSES[pred_idx]
        confidence = probabilities[pred_idx] * 100
        
        color = 'green' if pred_idx == true_label else 'red'
        title = f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.1f}%'
        axes[idx].set_title(title, fontsize=10, color=color, weight='bold')
    
    plt.tight_layout()
    plt.savefig('prediction_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# 결과 시각화 실행
# 실제 데이터가 준비되면 주석 해제하여 실행

# class_names = list(SKIN_DISEASE_CLASSES.values())

# # Confusion Matrix
# plot_confusion_matrix(results['confusion_matrix'], class_names)

# # 클래스별 성능
# plot_class_performance(results['classification_report'], class_names)

# # 예측 샘플 시각화
# visualize_predictions(model, data_df, CLASS_DESCRIPTIONS, num_samples=9)

## 6. 단일 이미지 테스트

In [ ]:
def test_single_image(model, image_path, class_descriptions, top_k=3):
    """
    단일 이미지에 대한 진단 수행
    
    Args:
        model: DermLIPModel 인스턴스
        image_path: 이미지 경로
        class_descriptions: 클래스 설명 리스트
        top_k: 상위 k개 예측 표시
    """
    # 이미지 로드
    image = Image.open(image_path).convert('RGB')
    
    # 예측
    pred_idx, probabilities = model.predict(image, class_descriptions)
    
    # 상위 k개 예측
    top_k_indices = np.argsort(probabilities)[::-1][:top_k]
    
    # 결과 출력
    print(f'\n이미지: {image_path}')
    print(f'\n상위 {top_k}개 예측:')
    print('-' * 50)
    for i, idx in enumerate(top_k_indices, 1):
        class_name = SKIN_DISEASE_CLASSES[idx]
        confidence = probabilities[idx] * 100
        print(f'{i}. {class_name:25s} : {confidence:6.2f}%')
    
    # 시각화
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 이미지 표시
    ax1.imshow(image)
    ax1.axis('off')
    ax1.set_title('입력 이미지', fontsize=14, pad=10)
    
    # 예측 확률 표시
    class_names = [SKIN_DISEASE_CLASSES[i] for i in range(len(SKIN_DISEASE_CLASSES))]
    y_pos = np.arange(len(class_names))
    
    colors = ['green' if i == pred_idx else 'skyblue' for i in range(len(class_names))]
    ax2.barh(y_pos, probabilities * 100, color=colors, alpha=0.8)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(class_names)
    ax2.set_xlabel('확률 (%)', fontsize=12)
    ax2.set_title('클래스별 예측 확률', fontsize=14, pad=10)
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('single_prediction.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# 단일 이미지 테스트 예시
# test_image_path = 'path/to/your/test/image.jpg'  # 테스트할 이미지 경로로 변경
# test_single_image(model, test_image_path, CLASS_DESCRIPTIONS, top_k=5)

## 7. 평가 결과 저장

In [ ]:
# 평가 결과를 JSON 파일로 저장
# import json

# results_summary = {
#     'model_name': model.model_name,
#     'accuracy': float(results['accuracy']),
#     'precision': float(results['precision']),
#     'recall': float(results['recall']),
#     'f1_score': float(results['f1_score']),
#     'num_samples': len(data_df),
#     'num_classes': len(SKIN_DISEASE_CLASSES),
#     'class_names': list(SKIN_DISEASE_CLASSES.values())
# }

# with open('evaluation_results.json', 'w', encoding='utf-8') as f:
#     json.dump(results_summary, f, indent=2, ensure_ascii=False)

# print('평가 결과가 evaluation_results.json에 저장되었습니다.')

## 8. 사용 가이드

### 데이터 준비

1. **이미지 데이터 준비**: 피부 질환 이미지를 수집하고 정리합니다.

2. **레이블 정보**: 각 이미지에 대한 정확한 진단 레이블을 준비합니다.

3. **데이터 포맷**:
   - CSV 파일: `image_path`, `label` 컬럼 포함
   - 또는 폴더 구조: `data/images/{class_name}/{image_file}`

### 실행 순서

1. **라이브러리 설치** (셀 1-2 실행)
2. **모델 로드** (셀 3-4 실행)
3. **데이터 로드** (셀 5-7 실행, 실제 데이터 경로로 수정)
4. **평가 실행** (셀 8-9 실행)
5. **결과 시각화** (셀 10-11 실행)

### 참고사항

- **GPU 권장**: CUDA 지원 GPU가 있으면 훨씬 빠른 추론이 가능합니다.
- **메모리**: 대용량 데이터셋의 경우 배치 크기를 조정하세요.
- **클래스 설명**: 더 상세하고 임상적인 설명을 사용하면 정확도가 향상될 수 있습니다.

### 추가 개선 방안

1. **Few-shot Learning**: 소량의 레이블된 데이터로 모델을 파인튜닝
2. **앙상블**: 여러 모델의 예측을 결합하여 성능 향상
3. **데이터 증강**: 이미지 회전, 플립 등을 통한 데이터 다양성 증가